In [20]:
# Setup

import pandas as pd
from pathlib import Path

In [21]:
# Import and read files

file22 = Path("data/unicorn_startups_sept_2022.csv")
df_2022 = pd.read_csv(file22, encoding="utf-8")

file23 = Path("data/unicorn_startups_july_2023.csv")
df_2023 = pd.read_csv(file23, encoding="utf-8")

file24 = Path("data/unicorn_startups_may_2024.xlsx")
df_2024 = pd.read_excel(file24)

In [22]:
# Make merged DF for three years

df_2022.rename(columns={"Valuation ($B)":"2022_Valuation"}, inplace=True)
df_2022.rename(columns={"Investors":"Select Investors"}, inplace=True)
df_2023.rename(columns={"Valuation ($B)":"2023_Valuation"}, inplace=True)
df_2024.rename(columns={"Valuation ($B)": "2024_Valuation"}, inplace=True)

first_merge = pd.merge(df_2022, df_2023[["Company", "2023_Valuation"]], on="Company", how="left")
merged_df = pd.merge(first_merge, df_2024[["Company", "2024_Valuation"]], on="Company", how="left")
merged_df = merged_df[["Company", "2022_Valuation", "2023_Valuation", "2024_Valuation", "Date Joined", "Country", 
                       "City", "Industry", "Select Investors"]]

merged_df.head()

,Company,2022_Valuation,2023_Valuation,2024_Valuation,Date Joined,Country,City,Industry,Select Investors
0,ByteDance,$140,$225,225.0,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,$127,$137,150.0,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,$100,$66,66.0,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,$95,$50,65.0,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Canva,$40,$40,25.4,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [23]:
# Drop missing values

merged_df.dropna(subset=["2022_Valuation", "2023_Valuation", "2024_Valuation", "Country"], inplace=True)

merged_df = merged_df.reset_index(drop=True)

# Convert valuation columns to float

merged_df["2022_Valuation"] = merged_df["2022_Valuation"].str.replace("$", "").astype(float)
merged_df["2023_Valuation"] = merged_df["2023_Valuation"].str.replace("$", "").astype(float)

# Calculate overall growth and CAGR (Compound Annual Growth Rate)

merged_df["Overall_Growth"] = merged_df["2024_Valuation"] - merged_df["2022_Valuation"]
merged_df["Overall_Growth"] = merged_df["Overall_Growth"].map("{:,.2f}".format)
merged_df["Overall_Growth"] = merged_df["Overall_Growth"].astype(float)

merged_df["CAGR (%)"] = ((merged_df["2024_Valuation"]/merged_df["2022_Valuation"])**(1/2) - 1) * 100
merged_df["CAGR (%)"] = merged_df["CAGR (%)"].map("{:,.2f}".format)
merged_df["CAGR (%)"] = merged_df["CAGR (%)"].astype(float)

merged_df.to_csv("data/merged_df.csv", header=True)

merged_df

,Company,2022_Valuation,2023_Valuation,2024_Valuation,Date Joined,Country,City,Industry,Select Investors,Overall_Growth,CAGR (%)
0,ByteDance,140.0,225.0,225.0,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S...",85.0,26.77
1,SpaceX,127.0,137.0,150.0,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen...",23.0,8.68
2,SHEIN,100.0,66.0,66.0,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China...",-34.0,-18.76
3,Stripe,95.0,50.0,65.0,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG",-30.0,-17.28
4,Canva,40.0,40.0,25.4,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat...",-14.6,-20.31
...,...,...,...,...,...,...,...,...,...,...,...
1066,LeadSquared,1.0,1.0,1.0,6/21/2022,India,Bengaluru,Internet software & services,"Gaja Capital Partners, Stakeboat Capital, West...",0.0,0.00
1067,FourKites,1.0,1.0,1.0,6/21/2022,United States,Chicago,"Supply chain, logistics, & delivery","Hyde Park Venture Partners, Bain Capital Ventu...",0.0,0.00
1068,VulcanForms,1.0,1.0,1.0,7/5/2022,United States,Burlington,"Supply chain, logistics, & delivery","Eclipse Ventures, D1 Capital Partners, Industr...",0.0,0.00
1069,SingleStore,1.0,1.0,1.0,7/12/2022,United States,San Francisco,Data management & analytics,"Google Ventures, Accel, Data Collective",0.0,0.00
